In [ ]:
import pandas as pd
import os
import re 
import json
import hdmf_zarr
from hdmf.common import DynamicTable
import numpy as np
from typing import List, Dict, Optional, Tuple
import glob 

def get_sessions_of_interest(summary_path: str, experiment_filter: str = 'NPUltra_psychedelics', 
                           upload_filter: str = 'yes') -> List[str]:
    """
    Generate list of sessions that meet specified criteria from recording summary.
    
    Args:
        summary_path: Path to NPUltra_recording_summary.xlsx
        experiment_filter: Experiment type to filter for
        upload_filter: Code Ocean Upload status to filter for
    
    Returns:
        List of sessions that match the criteria 
    """
    recording_summary_table = pd.read_excel(summary_path)
    
    filtered_sessions = recording_summary_table[
        (recording_summary_table['experiment'] == experiment_filter) &
        (recording_summary_table['uploaded to CO'] == upload_filter)
    ]
    
    session_list = filtered_sessions['session'].tolist()
    print(f"Found {len(session_list)} sessions matching criteria")
    
    return session_list

def extract_analysis_table(session_folder: str) -> Optional[pd.DataFrame]:
    # Optional indicates that function can return None or pd.DataFrame 
    """
    Extract [postprocessed] units table from the session folder. This table should be renamed to analysis_table to avoid 
    confusion with the existing units table in the NWB file. 
    
    Args:
        session_folder: Path to session directory
    
    Returns:
        DataFrame with analysis data or None if not found
    """
    analysis_table_path = os.path.join(session_folder, "processed_data", "units")
    
    if not os.path.exists(analysis_table_path):
        print(f"Units folder does not exist in {session_folder}")
        return None
    
    # Find units table pickle (exclude units_epochs)
    analysis_files = [f for f in os.listdir(analysis_table_path) 
                   if 'units_epoch' in f and 'units_epochs' not in f]
    
    if not analysis_files:
        print(f"No units table files found in {analysis_table_path}")
        return None
    
    analysis_table_file = os.path.join(analysis_table_path, analysis_files[0])
    analysis_table = pd.read_pickle(analysis_table_file)
    print(f"Loaded units table from {analysis_table_file}")

    # Modify units_table to match final analysis_table format 
    # Remove unnecessary columns 
    exclude_columns = ['sessionID', 'mouseID', 'genotype', 'experiment', 'recording', 'recording_date', 'probe_config', 'spike_times', '10ms_pulse_psth', '200ms_pulse_psth']    
    analysis_table = analysis_table.drop(columns=exclude_columns, errors='ignore')

    # Rename columns 
    analysis_table = analysis_table.rename(columns={
        'unitID': 'ks_unit_id', 
        'amp': 'amplitude', 
        'dur': 'duration', 
        'PTR': 'peak_trough_ratio',
        'prePTR': 'pre_peak_trough_ratio',
        'repol_slope': 'repolarization_slope',
        'recov_slope': 'recovery_slope',
        'ct': 'cell_type'
    })

    return analysis_table

def extract_opto_stim_table(session_folder: str) -> Optional[pd.DataFrame]:
    """
    Extract stimulus table from session folder and add stimulus information from metadata.
    
    Args:
        session_folder: Path to session directory
    
    Returns:
        DataFrame with stimulus data or None if not found
    """
    stim_path = os.path.join(session_folder, "processed_data", "stim")
    
    if not os.path.exists(stim_path):
        print(f"Stimulus folder does not exist in {session_folder}")
        return None
    
    # Initialize stimulus table
    opto_stim_table = pd.DataFrame(columns=['start_time', 'stop_time', 'power', 'stim_name', 
                                     'probe', 'wavelength'])
    
    # Load stimulus table from opto_stim_df.csv
    opto_stim_table_path = os.path.join(stim_path, "opto_stim_df.csv")
    if not os.path.exists(opto_stim_table_path):
        print(f"No stimulus table files found in {stim_path}")
        return None
    
    opto_stim_df = pd.read_csv(opto_stim_table_path)
    print(f"Loaded stimulus table from {opto_stim_table_path}")
    
    # Populate stimulus table
    opto_stim_table['start_time'] = opto_stim_df['stim_on']
    opto_stim_table['stop_time'] = opto_stim_df['stim_off']
    opto_stim_table['stim_name'] = opto_stim_df['epoch_label']
    opto_stim_table['probe'] = opto_stim_df['probe']
    
    # Extract more stimulus information from session.json metadata 
    metadata_path = os.path.join(session_folder, "metadata", "session.json")
    if os.path.exists(metadata_path):
        with open(metadata_path, 'r') as f:
            metadata = json.load(f)
            
        # Extract wavelength and power from first stimulus epoch light_source_config 
        wavelength = None
        power = None
        for epoch in metadata.get('stimulus_epochs', []):
            light_config = epoch.get('light_source_config')
            if light_config is not None:
                wavelength = light_config.get('wavelength')
                power = light_config.get('laser_power')
                break # Only store first epoch's config as the rest are redundant 
        
        opto_stim_table['wavelength'] = wavelength
        opto_stim_table['power'] = power
    
    return opto_stim_table

def extract_visual_stim_table(session_folder: str) -> Optional[pd.DataFrame]:
    """
    Extract visual stimulus (RFMapping) table from session folder and apply modifications.
    
    Args:
        session_folder: Path to session directory
    
    Returns:
        DataFrame with stimulus data or None if not found
    """
    stim_path = os.path.join(session_folder, "processed_data", "stim")
    
    if not os.path.exists(stim_path):
        print(f"Stimulus folder does not exist in {session_folder}")
        return None
    
    # Load stimulus table from opto_stim_df.csv
    visual_stim_table_path = os.path.join(stim_path, "RFMapping_stim_df.csv")
    if not os.path.exists(visual_stim_table_path):
        print(f"No visual stimulus table files found in {stim_path}")
        return None
    
    visual_stim_table = pd.read_csv(visual_stim_table_path)
    print(f"Loaded visual stimulus table from {visual_stim_table_path}")

    # Modify visual stimulus table 
    visual_stim_table = visual_stim_table.rename(columns={'epoch_label': 'stim_name'}) # Rename epoch label column to stim_name 

    return visual_stim_table

def create_epoch_table(session_folder: str) -> Optional[pd.DataFrame]:
    """
    Create epoch table from stimulus epoch files and metadata.
    
    Args:
        session_folder: Path to session directory
    
    Returns:
        DataFrame with epoch information
    """
    epoch_path = os.path.join(session_folder, "processed_data", "stim")
    
    if not os.path.exists(epoch_path):
        print(f"Stimulus folder does not exist in {session_folder}")
        return None
    
    # Process each CSV file that contains trial by trial timing information for each stimulus epoch
    all_epoch_data = []
    epoch_files = [f for f in os.listdir(epoch_path) if f.endswith('.csv')]
    
    if not epoch_files:
        print(f"No CSV files found in {epoch_path}")
        return None
    
    # Each CSV corresponds to a different stimulus epoch type 
    for epoch_file in epoch_files:
        file_path = os.path.join(epoch_path, epoch_file)
        
        # Read required columns, handle missing stim_off 
        # Only the OptoTagging epochs have a stim_off column, others do not 
        try:
            temp_df = pd.read_csv(file_path, usecols=['stim_on', 'stim_off', 'epoch_label'])
        except ValueError:
            temp_df = pd.read_csv(file_path, usecols=['stim_on', 'epoch_label'])
            temp_df['stim_off'] = pd.NA
        
        all_epoch_data.append(temp_df)
    
    # Combine trial-by-trial information across all stimulus epochs
    combined_df = pd.concat(all_epoch_data, ignore_index=True)
    
    # Group combined dataframe by epoch_label to get start/stop times for the entire epoch 
    epoch_summary = combined_df.groupby('epoch_label').apply(
        lambda group: pd.Series({
            'start_time': group['stim_on'].min(),
            'stop_time': group['stim_off'].max() if 'OptoTagging' in group.name else group['stim_on'].max()
        }) 
    ).reset_index() # Use max for stim_off only if it exists, otherwise use max of stim_on
    
    # Rename columns 
    epoch_table = epoch_summary.rename(columns={'epoch_label': 'stim_name'})
    
    # Sort epochs by start_time 
    epoch_table = epoch_table.sort_values(by='start_time').reset_index(drop=True)
    
    return epoch_table

def add_injection_epoch(epoch_table: pd.DataFrame, session_folder: str) -> pd.DataFrame:
    """
    Add injection epoch between OptoTagging_0 and Spontaneous_1. 
    
    Args:
        epoch_table: Existing epoch table
        session_folder: Path to session directory
    
    Returns:
        Modified epoch table with injection epoch added
    """
    metadata_path = os.path.join(session_folder, "metadata", "session.json")
    
    if not os.path.exists(metadata_path):
        print(f"Metadata file does not exist in {metadata_path}")
        return epoch_table
    
    with open(metadata_path, 'r') as f:
        metadata = json.load(f)
    
    # Determine injection type from OptoTagging stimulus epoch note 
    injection_name = "None"  # Default if missing info in metadata 
    for epoch in metadata.get('stimulus_epochs', []): # If saline or psilocybin injection noted, add Injection epoch 
        if epoch.get('stimulus_name') == 'OptoTagging':
            notes = epoch.get('notes', '')
            if 'Saline' in notes:
                injection_name = "Injection"
            elif 'Psilocybin' in notes:
                injection_name = "Injection"
            break
    
    # Get start and stop times for new epoch based on the timing of Optotagging_0 and Spontaneous_1 epochs 
    opto_0_stop = None
    spont_1_start = None
    
    for idx, row in epoch_table.iterrows():
        if row['stim_name'] == 'OptoTagging_0': # Get value of stop_time for OptoTagging_0
            opto_0_stop = row['stop_time']
        elif row['stim_name'] == 'Spontaneous_1': # Get value of start_time for Spontaneous_1 
            spont_1_start = row['start_time']
    
    if opto_0_stop is not None and spont_1_start is not None:
        # Create injection epoch with time buffer 
        injection_start = opto_0_stop + 0.01
        injection_end = spont_1_start - 0.01
        
        # Identify the correct index in epoch_table to insert the new epoch (after Optotagging_0)
        insert_idx = None
        for idx, row in epoch_table.iterrows():
            if row['stim_name'] == 'OptoTagging_0':
                insert_idx = idx + 1
                break
        
        # Fill out the new row with injection information 
        new_row = pd.DataFrame({
            'stim_name': [injection_name],
            'start_time': [injection_start],
            'stop_time': [injection_end]
        })
        
        # Insert the new row at the correct index into the epoch_table 
        if insert_idx is not None:
            epoch_table = pd.concat([
                epoch_table.iloc[:insert_idx], # Keep rows before the insertion point 
                new_row, 
                epoch_table.iloc[insert_idx:] # Keep rows after the insertion point 
            ], ignore_index=True)
            print(f"Added {injection_name} epoch from {injection_start} to {injection_end}")
    
    return epoch_table

def convert_dataframe_to_dynamic_table(df: pd.DataFrame, table_name: str) -> DynamicTable:
    """
    Convert pandas DataFrame to DynamicTable object compatible with NWB files. 
    
    Args:
        df: Input DataFrame
        table_name: Name for the dynamic table
    
    Returns:
        DynamicTable object
    """
    dynamic_table = DynamicTable.from_dataframe(
        name=table_name.lower().replace(' ', '_'),
        df=df
    )
    # name ensures the input is lowercase and has underscores  
    print(f"Created DynamicTable '{table_name}' with {len(df)} rows and {len(df.columns)} columns")
    return dynamic_table

def modify_nwb_file(original_path: str, new_path: str, analysis_table: Optional[pd.DataFrame] = None, opto_stim_table: Optional[pd.DataFrame] = None, 
                    visual_stim_table: Optional[pd.DataFrame] = None, epoch_table: Optional[pd.DataFrame] = None) -> None:
    """
    Modify existing NWB file with new tables and save to new location. 
    Tables are pandas DataFrames that will be converted to DynamicTables. 
    
    Args:
        original_path: Path to input NWB file
        new_path: Path for output NWB file
        analysis_table: Analysis table to add
        opto_stim_table: Optogenetic Stimulus table to add
        visual_stim_table: Visual Stimulus (RFMapping) table to add 
        epoch_table: Epoch table to add
    """
    # Create directory for new file
    os.makedirs(os.path.dirname(new_path), exist_ok=True)
    
    # Open existing NWB file
    io = hdmf_zarr.NWBZarrIO(original_path, mode="r")
    nwbfile = io.read()
    
    # # Add modified units table 
    # if modified_units_table is not None: 
    #     print("Processing units table...")
    #     new_units_table = DynamicTable.from_dataframe(
    #         name='units'
    #         df=modified_units_table
    #     )
    #     nwbfile.units = new_units_table 
    #     print(f"Added units table with {len(modified_units_table)} rows")

    # Add analysis table
    if analysis_table is not None:
        print("Processing analysis table...")
        try:
            analysis_dynamic_table = convert_dataframe_to_dynamic_table(
                df=analysis_table,
                table_name="analysis_table"
            )
            nwbfile.add_analysis(analysis_dynamic_table)
            print(f"Added analysis table with {len(analysis_table)} rows")
        except Exception as e:
            print(f"Failed to add analysis table: {e}")
            raise
    
    # Add optogenetic stimulus table
    if opto_stim_table is not None:
        print("Processing optogenetic stimulus table...")
        try:
            stimulus_dynamic_table = convert_dataframe_to_dynamic_table(
                df=opto_stim_table,
                table_name="photostim_table"
            )
            nwbfile.add_stimulus(stimulus_dynamic_table)
            print(f"Added stimulus table with {len(opto_stim_table)} rows")
        except Exception as e:
            print(f"Failed to add stimulus table: {e}")
            raise

    # Add visual stimulus table
    if visual_stim_table is not None:
        print("Processing visual stimulus table...")
        try:
            stimulus_dynamic_table = convert_dataframe_to_dynamic_table(
                df=visual_stim_table,
                table_name="visualstim_table"
            )
            nwbfile.add_stimulus(stimulus_dynamic_table)
            print(f"Added visual stimulus table with {len(visual_stim_table)} rows")
        except Exception as e:
            print(f"Failed to add viual stimulus table: {e}")
            raise
    
    # Add epoch table
    if epoch_table is not None:
        print("Processing epoch table...")
        try: # Constructs epoch table row by row from DataFrame  
            for idx, row in epoch_table.iterrows():
                tags = str(row['stim_name']) if 'stim_name' in row and pd.notna(row['stim_name']) else []
                nwbfile.add_epoch(
                    start_time=float(row['start_time']),
                    stop_time=float(row['stop_time']), 
                    tags=tags
                )
            print(f"Added {len(epoch_table)} epochs")
        except Exception as e:
            print(f"Failed to add epoch table: {e}")
            raise
    
    # Write modified NWB to new file 
    with hdmf_zarr.NWBZarrIO(new_path, mode='w') as export_io:
        export_io.export(src_io=io, nwbfile=nwbfile)
    
    io.close()
    print(f"Saved to: {new_path}")

def process_single_session(session_name: str, base_path: str, output_path: str) -> None:
    """
    Process a single session through the complete pipeline. Finds correct file paths based on session_name for all necessary directories and files. 
    
    Args:
        session_name: Name of the session to process
        base_path: Base path containing session folders
        output_path: Output directory for modified NWB files
    """
    session_folder = os.path.join(base_path, session_name)
    print(f"\nProcessing session: {session_name}")
    
    if not os.path.exists(session_folder):
        print(f"Session folder {session_folder} does not exist")
        return
    
    # Extract all tables
    analysis_table = extract_analysis_table(session_folder)
    opto_stim_table = extract_opto_stim_table(session_folder)
    visual_stim_table = extract_visual_stim_table(session_folder)
    epoch_table = create_epoch_table(session_folder)
    
    # Add injection epoch if epoch table exists
    if epoch_table is not None:
        epoch_table = add_injection_epoch(epoch_table, session_folder)
    
    # Find original NWB file
    nwb_search_path = os.path.join(session_folder, f"*{session_name.split('_')[1]}*")
    
    # First, find directories containing the session name
    cleaned_session_name = session_name[0].replace('-', '')
    # Within the session folder, search for directories that contain the cleaned session name
    session_dirs = glob.glob(os.path.join(session_folder, f"*{cleaned_session_name}*")) 
    if not session_dirs:
        print(f"No directory containing '{cleaned_session_name}' found in {session_folder}")

    # Then search for NWB files containing the session name within those directories
    nwb_files = []
    for session_dir in session_dirs:
        # instead, search for a pattern that ends in .nwb 
        nwb_pattern = os.path.join(session_dir, f"*.nwb")
        # Get full file path to nwb file 
        nwb_files.extend(glob.glob(nwb_pattern))
        # Get nwb file name to match for output file 
        nwb_files.extend([f for f in os.listdir(session_dir) if f.endswith('.nwb')])

    if not nwb_files:
        print(f"No NWB file containing '{session_name}' found in session directories")

    original_nwb = nwb_files[0]  # Take the first match
    print(f"Found NWB file: {original_nwb}")
    new_nwb = os.path.join(output_path, nwb_files[1])
    
    # Modify NWB file
    modify_nwb_file(
        original_path=original_nwb,
        new_path=new_nwb,
        analysis_table=analysis_table,
        opto_stim_table=opto_stim_table,
        visual_stim_table=visual_stim_table,
        epoch_table=epoch_table
    )

In [22]:
# Example usage: 
filtered_session_list = get_sessions_of_interest(
    summary_path = "/Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/NPUltra_recording_summary.xlsx",
    experiment_filter = "NPUltra_psychedelics", 
    upload_filter = 'yes'
    )

process_single_session(
    session_name = filtered_session_list[0],  # Process one session as an example 
    base_path = '/Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/',
    output_path = '/Volumes/scratch/suyee.lee'
)

Found 12 sessions matching criteria

Processing session: 2024-05-14_714527
Loaded units table from /Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/2024-05-14_714527/processed_data/units/2024-05-14_714527_1_units_epoch.pkl
Loaded stimulus table from /Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/2024-05-14_714527/processed_data/stim/opto_stim_df.csv
Loaded visual stimulus table from /Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/2024-05-14_714527/processed_data/stim/RFMapping_stim_df.csv


/var/folders/1f/h0zlr4c90tg4sf8rv_22d7v80000gp/T/ipykernel_63449/2341047976.py:255: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_df = pd.concat(all_epoch_data, ignore_index=True)
/var/folders/1f/h0zlr4c90tg4sf8rv_22d7v80000gp/T/ipykernel_63449/2341047976.py:258: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  epoch_summary = combined_df.groupby('epoch_label').apply(


Added Injection_Time epoch from 2714.53929 to 3058.6834599999997
Found NWB file: /Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/2024-05-14_714527/20240514_714527_1/ecephys_714527_2024-05-14_13-22-07_experiment1_recording1.nwb


/Users/suyee.lee/np-ultra-psychedelics/.venv/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


Processing analysis table...
Created DynamicTable 'analysis_table' with 350 rows and 44 columns
Added analysis table with 350 rows
Processing optogenetic stimulus table...
Created DynamicTable 'photostim_table' with 600 rows and 6 columns
Added stimulus table with 600 rows
Processing visual stimulus table...
Created DynamicTable 'visualstim_table' with 8131 rows and 6 columns
Added visual stimulus table with 8131 rows
Processing epoch table...
Added 13 epochs


/Users/suyee.lee/np-ultra-psychedelics/.venv/lib/python3.12/site-packages/zarr/storage.py:455: FutureWarning: missing object_codec for object array; this will raise a ValueError in version 3.0
  _init_array_metadata(


Saved to: /Volumes/scratch/suyee.lee/ecephys_714527_2024-05-14_13-22-07_experiment1_recording1.nwb


In [23]:
# Check outputs of new nwb file 
# 
new_nwb = '/Volumes/scratch/suyee.lee/ecephys_714527_2024-05-14_13-22-07_experiment1_recording1.nwb'

with hdmf_zarr.NWBZarrIO(new_nwb, mode='r') as io:
    new_nwbfile = io.read()
    print("Loaded new NWB file:", new_nwb)
    print("Processing modules:", list(new_nwbfile.processing.keys()))
    print("Stimulus presentations and epochs:", list(new_nwbfile.stimulus.keys()) if hasattr(new_nwbfile, 'stimulus') else "None")
    print("Epochs:", new_nwbfile.epochs)  # Should show the added epochs
    print("Trials:", new_nwbfile.trials)  # Should show the added trials


Loaded new NWB file: /Volumes/scratch/suyee.lee/ecephys_714527_2024-05-14_13-22-07_experiment1_recording1.nwb
Processing modules: []
Stimulus presentations and epochs: ['photostim_table', 'visualstim_table']
Epochs: epochs pynwb.epoch.TimeIntervals at 0x4426486352
Fields:
  colnames: ['start_time' 'stop_time' 'tags']
  columns: (
    start_time <class 'hdmf.common.table.VectorData'>,
    stop_time <class 'hdmf.common.table.VectorData'>,
    tags_index <class 'hdmf.common.table.VectorIndex'>,
    tags <class 'hdmf.common.table.VectorData'>
  )
  description: experimental epochs
  id: id <class 'hdmf.common.table.ElementIdentifiers'>

Trials: None


In [24]:
new_nwbfile.stimulus["photostim_table"]

,start_time,stop_time,power,stim_name,probe,wavelength
id,,,,,,
0,2473.89271,2473.90471,13.5,OptoTagging_0,B,488
1,2475.16007,2475.36207,13.5,OptoTagging_0,F,488
2,2476.36221,2476.37421,13.5,OptoTagging_0,D,488
3,2477.49455,2477.50656,13.5,OptoTagging_0,F,488


In [ ]:
new_nwbfile

In [25]:
new_nwbfile.epochs

,start_time,stop_time,tags
id,,,
0,47.95425,1249.948000,[S]
1,1259.78593,2461.428177,[R]
2,2473.89271,2714.529290,[O]
3,2714.53929,3058.683460,[I]


In [2]:
filtered_session_list = get_sessions_of_interest(
    summary_path = "/Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/NPUltra_recording_summary.xlsx",
    experiment_filter = "NPUltra_psychedelics", 
    upload_filter = 'yes'
    )

Found 12 sessions matching criteria


In [ ]:
import re 

session_folder = '/Volumes/scratch/andrew.shelton/NPUltra_data/raw_npultra_data/2024-05-14_714527'
session_name = filtered_session_list[0]
session_pattern = r'\d{4}-\d{2}-\d{2}_(\d{6})'
match = re.search(session_pattern, session_name)
subject_id = match.group(1) 
nwb_path = [] 

# Search for directories containing the subject ID 
subject_dir = glob.glob(os.path.join(session_folder, f"*{subject_id}*"))   

if not subject_dir:
    print(f"No directory containing '{subject_id}' found in {session_folder}")

nwb_pattern = os.path.join(subject_dir[0], "*.nwb")
nwb_path.extend(glob.glob(nwb_pattern)) 

if not os.path.exists(nwb_path[0]):
    print(f"NWB file {nwb_path[0]} does not exist")

# Load nwb file using 
# 
with hdmf_zarr.NWBZarrIO(nwb_path[0], mode='r') as io:
    nwbfile = io.read()
    # Get units table 
    units_table = nwbfile.units.to_dataframe()

# Modify units table 
modified_units_table = units_table.copy()  # Create a copy to avoid modifying the original
modified_units_table = modified_units_table.rename(columns={
    'ks_unit_id': 'unitID'
})

# Reorganize columns alphabetically 
modified_units_table = modified_units_table.reindex(sorted(modified_units_table.columns), axis=1)
# Reorganize columns again by putting unitID first 
unitID_col = modified_units_table.pop('unitID') # Remove the column 
modified_units_table.insert(0, 'unitID', unitID_col) # Insert it back at the beginning 

# Save modified units table 
modified_units_table 


/Users/suyee.lee/np-ultra-psychedelics/.venv/lib/python3.12/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


,unitID,amplitude,amplitude_cutoff,amplitude_cv_median,amplitude_cv_range,amplitude_median,d_prime,decoder_label,decoder_probability,default_qc,...,spike_times,spread,sync_spike_2,sync_spike_4,sync_spike_8,unit_name,velocity_above,velocity_below,waveform_mean,waveform_sd
id,,,,,,,,,,,,,,,,,,,,,
0,0.0,63.490002,0.000093,NaN,NaN,58.499996,6.549358,mua,0.73,False,...,"[77.12925173733235, 78.55781998509673, 80.4399...",180.0,0.006778,0.000000,0.0,0a74cc68-d695-4965-bb8b-23464bb35d0b,497.598613,-506.297505,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,1.0,89.129997,0.000205,NaN,NaN,341.639980,3.411257,mua,0.65,False,...,"[69.72397687472352, 89.40936532860391, 91.5384...",180.0,0.006891,0.000000,0.0,a14cbbc8-3f2a-46d7-b136-9f668e983bfb,410.917576,-914.033536,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,2.0,72.279999,0.000040,NaN,NaN,119.340000,3.580504,mua,0.61,True,...,"[68.04330834794193, 68.63350900115876, 68.7390...",156.0,0.002310,0.000000,0.0,7d416677-ea24-4741-8f3c-4d9c789a17df,383.350401,-877.514461,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3.0,43.630001,0.000197,NaN,NaN,42.120000,2.240950,mua,0.85,False,...,"[178.2583544572817, 299.6283704877514, 217.851...",192.0,0.022500,0.000000,0.0,8ce06fb5-4c13-4b45-818a-aa44683e1878,NaN,-360.307571,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,4.0,135.059998,0.000006,NaN,NaN,140.400000,3.943305,sua,0.81,True,...,"[155.50719504325926, 205.00218658902662, 59.80...",84.0,0.005011,0.000000,0.0,d0784418-201b-4b79-896c-c0395e334742,373.356109,NaN,"[[-0.38843998312950134, 0.0, -0.58032029867172...","[[15.311971664428711, 0.0, 15.367671966552734,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
417,108.0,103.370003,0.000314,NaN,NaN,109.979996,2.168903,mua,0.80,False,...,"[108.06291653052462, 154.45717152233937, 150.7...",186.0,0.056713,0.000000,0.0,346d2056-da4d-43d2-aa57-ea68c821f8b5,873.705167,-1511.933770,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
418,109.0,125.080002,0.016413,NaN,NaN,194.219990,5.486762,mua,0.55,False,...,"[154.85214677357945, 26.9456888292128, 152.535...",102.0,0.032759,0.000000,0.0,5400e58a-1945-41c6-8af5-a2373bc369de,NaN,NaN,"[[-2.2230000495910645, 0.0, -2.924999237060547...","[[20.50239372253418, 0.0, 20.178279876708984, ..."
419,110.0,114.519997,0.000068,NaN,NaN,124.020000,3.290996,sua,0.61,True,...,"[40.728704313631226, 35.788630296656095, 32.58...",174.0,0.015123,0.000022,0.0,45dfab7e-6d81-4448-b906-730520123f75,720.879701,-1053.961902,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [23]:
nwbfile.units

units pynwb.misc.Units at 0x4969571696
Fields:
  colnames: ['spike_times' 'electrodes' 'waveform_mean' 'waveform_sd' 'unit_name'
 'drift_ptp' 'half_width' 'sync_spike_4' 'sync_spike_2' 'presence_ratio'
 'num_spikes' 'amplitude_cv_median' 'firing_range' 'decoder_probability'
 'nn_hit_rate' 'velocity_below' 'sliding_rp_violation' 'exp_decay'
 'num_positive_peaks' 'amplitude_cutoff' 'velocity_above' 'amplitude'
 'estimated_x' 'isi_violations_ratio' 'sync_spike_8'
 'isi_violations_count' 'decoder_label' 'estimated_y' 'device_name'
 'peak_trough_ratio' 'ks_unit_id' 'depth' 'l_ratio' 'estimated_z'
 'd_prime' 'rp_violations' 'drift_mad' 'snr' 'silhouette' 'nn_miss_rate'
 'isolation_distance' 'amplitude_cv_range' 'drift_std'
 'num_negative_peaks' 'recovery_slope' 'repolarization_slope'
 'peak_to_valley' 'rp_contamination' 'spread' 'amplitude_median'
 'default_qc' 'firing_rate']
  columns: (
    spike_times_index <class 'hdmf.common.table.VectorIndex'>,
    spike_times <class 'hdmf.common.table.VectorData'>,
    electrodes_index <class 'hdmf.common.table.VectorIndex'>,
    electrodes <class 'hdmf.common.table.DynamicTableRegion'>,
    waveform_mean <class 'hdmf.common.table.VectorData'>,
    waveform_sd <class 'hdmf.common.table.VectorData'>,
    unit_name <class 'hdmf.common.table.VectorData'>,
    drift_ptp <class 'hdmf.common.table.VectorData'>,
    half_width <class 'hdmf.common.table.VectorData'>,
    sync_spike_4 <class 'hdmf.common.table.VectorData'>,
    sync_spike_2 <class 'hdmf.common.table.VectorData'>,
    presence_ratio <class 'hdmf.common.table.VectorData'>,
    num_spikes <class 'hdmf.common.table.VectorData'>,
    amplitude_cv_median <class 'hdmf.common.table.VectorData'>,
    firing_range <class 'hdmf.common.table.VectorData'>,
    decoder_probability <class 'hdmf.common.table.VectorData'>,
    nn_hit_rate <class 'hdmf.common.table.VectorData'>,
    velocity_below <class 'hdmf.common.table.VectorData'>,
    sliding_rp_violation <class 'hdmf.common.table.VectorData'>,
    exp_decay <class 'hdmf.common.table.VectorData'>,
    num_positive_peaks <class 'hdmf.common.table.VectorData'>,
    amplitude_cutoff <class 'hdmf.common.table.VectorData'>,
    velocity_above <class 'hdmf.common.table.VectorData'>,
    amplitude <class 'hdmf.common.table.VectorData'>,
    estimated_x <class 'hdmf.common.table.VectorData'>,
    isi_violations_ratio <class 'hdmf.common.table.VectorData'>,
    sync_spike_8 <class 'hdmf.common.table.VectorData'>,
    isi_violations_count <class 'hdmf.common.table.VectorData'>,
    decoder_label <class 'hdmf.common.table.VectorData'>,
    estimated_y <class 'hdmf.common.table.VectorData'>,
    device_name <class 'hdmf.common.table.VectorData'>,
    peak_trough_ratio <class 'hdmf.common.table.VectorData'>,
    ks_unit_id <class 'hdmf.common.table.VectorData'>,
    depth <class 'hdmf.common.table.VectorData'>,
    l_ratio <class 'hdmf.common.table.VectorData'>,
    estimated_z <class 'hdmf.common.table.VectorData'>,
    d_prime <class 'hdmf.common.table.VectorData'>,
    rp_violations <class 'hdmf.common.table.VectorData'>,
    drift_mad <class 'hdmf.common.table.VectorData'>,
    snr <class 'hdmf.common.table.VectorData'>,
    silhouette <class 'hdmf.common.table.VectorData'>,
    nn_miss_rate <class 'hdmf.common.table.VectorData'>,
    isolation_distance <class 'hdmf.common.table.VectorData'>,
    amplitude_cv_range <class 'hdmf.common.table.VectorData'>,
    drift_std <class 'hdmf.common.table.VectorData'>,
    num_negative_peaks <class 'hdmf.common.table.VectorData'>,
    recovery_slope <class 'hdmf.common.table.VectorData'>,
    repolarization_slope <class 'hdmf.common.table.VectorData'>,
    peak_to_valley <class 'hdmf.common.table.VectorData'>,
    rp_contamination <class 'hdmf.common.table.VectorData'>,
    spread <class 'hdmf.common.table.VectorData'>,
    amplitude_median <class 'hdmf.common.table.VectorData'>,
    default_qc <class 'hdmf.common.table.VectorData'>,
    firing_rate <class 'hdmf.comm